In [1]:
# https://www.johnsonmitchelld.com/2021/03/14/least-squares-gps.html
import sys, os, csv
from datetime import datetime, timezone, timedelta
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import navpy
from gnssutils.ephemeris_manager import EphemerisManager

parent_directory = os.path.split(os.getcwd())[0]
ephemeris_data_directory = os.path.join(parent_directory, 'Data/Fixed/')
sys.path.insert(0, parent_directory)

/tmp/ipykernel_4691/3513459166.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
def create_dataframes() -> tuple[pd.DataFrame, pd.DataFrame]:
    input_filepath = os.path.join('Data', 'Fixed', 'gnss_log_2024_04_13_19_51_17.txt')

    with open(input_filepath, 'r') as f:
        reader = csv.reader(f)
        for row in reader:
            if row[0][0] == '#':
                if 'Fix' in row[0]:
                    android_fixes = [row[1:]]
                elif 'Raw' in row[0]:
                    measurements = [row[1:]]
            else:
                if row[0] == 'Fix':
                    android_fixes.append(row[1:])
                elif row[0] == 'Raw':
                    measurements.append(row[1:])

    android_fixes = pd.DataFrame(android_fixes[1:], columns = android_fixes[0])
    measurements = pd.DataFrame(measurements[1:], columns = measurements[0])

    # Format satellite IDs
    measurements.loc[measurements['Svid'].str.len() == 1, 'Svid'] = '0' + measurements['Svid']
    measurements.loc[measurements['ConstellationType'] == '1', 'Constellation'] = 'G'
    measurements.loc[measurements['ConstellationType'] == '3', 'Constellation'] = 'R'
    measurements['SvName'] = measurements['Constellation'] + measurements['Svid']

    # Remove all non-GPS measurements
    measurements = measurements.loc[measurements['Constellation'] == 'G']

    # Convert columns to numeric representation
    measurements['Cn0DbHz'] = pd.to_numeric(measurements['Cn0DbHz'])
    measurements['TimeNanos'] = pd.to_numeric(measurements['TimeNanos'])
    measurements['FullBiasNanos'] = pd.to_numeric(measurements['FullBiasNanos'])
    measurements['ReceivedSvTimeNanos']  = pd.to_numeric(measurements['ReceivedSvTimeNanos'])
    measurements['PseudorangeRateMetersPerSecond'] = pd.to_numeric(measurements['PseudorangeRateMetersPerSecond'])
    measurements['ReceivedSvTimeUncertaintyNanos'] = pd.to_numeric(measurements['ReceivedSvTimeUncertaintyNanos'])

    # A few measurement values are not provided by all phones
    # We'll check for them and initialize them with zeros if missing
    if 'BiasNanos' in measurements.columns:
        measurements['BiasNanos'] = pd.to_numeric(measurements['BiasNanos'])
    else:
        measurements['BiasNanos'] = 0
    if 'TimeOffsetNanos' in measurements.columns:
        measurements['TimeOffsetNanos'] = pd.to_numeric(measurements['TimeOffsetNanos'])
    else:
        measurements['TimeOffsetNanos'] = 0

    return measurements, android_fixes

In [3]:
def timestamp_generation(measurements: pd.DataFrame) -> pd.DataFrame:
    measurements['GpsTimeNanos'] = measurements['TimeNanos'] - (measurements['FullBiasNanos'] - measurements['BiasNanos'])
    gpsepoch = datetime(1980, 1, 6, 0, 0, 0)
    measurements['UnixTime'] = pd.to_datetime(measurements['GpsTimeNanos'], utc = True, origin=gpsepoch)
    measurements['UnixTime'] = measurements['UnixTime']

    # Split data into measurement epochs
    measurements['Epoch'] = 0
    measurements.loc[measurements['UnixTime'] - measurements['UnixTime'].shift() > timedelta(milliseconds=200), 'Epoch'] = 1
    measurements['Epoch'] = measurements['Epoch'].cumsum()

    return measurements

In [4]:
def pseudorange_calculation(measurements: pd.DataFrame) -> pd.DataFrame:
    WEEKSEC = 604800
    LIGHTSPEED = 2.99792458e8

    # This should account for rollovers since it uses a week number specific to each measurement
    measurements['tRxGnssNanos'] = measurements['TimeNanos'] + measurements['TimeOffsetNanos'] - (measurements['FullBiasNanos'].iloc[0] + measurements['BiasNanos'].iloc[0])
    measurements['GpsWeekNumber'] = np.floor(1e-9 * measurements['tRxGnssNanos'] / WEEKSEC)
    measurements['tRxSeconds'] = 1e-9*measurements['tRxGnssNanos'] - WEEKSEC * measurements['GpsWeekNumber']
    measurements['tTxSeconds'] = 1e-9*(measurements['ReceivedSvTimeNanos'] + measurements['TimeOffsetNanos'])
    # Calculate pseudorange in seconds
    measurements['prSeconds'] = measurements['tRxSeconds'] - measurements['tTxSeconds']

    # Conver to meters
    measurements['PrM'] = LIGHTSPEED * measurements['prSeconds']
    measurements['PrSigmaM'] = LIGHTSPEED * 1e-9 * measurements['ReceivedSvTimeUncertaintyNanos']

    return measurements

In [5]:
def retrieve_ephemeris_data(measurements: pd.DataFrame, epoch: int) -> tuple:
    manager = EphemerisManager(r'Data/Fixed/')

    # epoch = 0
    # num_sats = 0
    one_epoch = measurements.loc[(measurements['Epoch'] == epoch) & (measurements['prSeconds'] < 0.1)].drop_duplicates(subset='SvName')
    timestamp = one_epoch.iloc[0]['UnixTime'].to_pydatetime(warn=False)
    one_epoch.set_index('SvName', inplace=True)
    # num_sats = len(one_epoch.index)
    # epoch += 1

    sats = one_epoch.index.unique().tolist()
    # print(type(timestamp))
    ephemeris = manager.get_ephemeris(timestamp, sats)
    # print(timestamp)
    # print(one_epoch[['UnixTime', 'tTxSeconds', 'GpsWeekNumber']])

    return ephemeris, one_epoch

In [6]:
def calculate_satellite_position(ephemeris, transmit_time) -> pd.DataFrame:
    mu = 3.986005e14
    OmegaDot_e = 7.2921151467e-5
    F = -4.442807633e-10
    sv_position = pd.DataFrame()
    sv_position['sv']= ephemeris.index
    sv_position.set_index('sv', inplace=True)
    sv_position['t_k'] = transmit_time - ephemeris['t_oe']
    A = ephemeris['sqrtA'].pow(2)
    n_0 = np.sqrt(mu / A.pow(3))
    n = n_0 + ephemeris['deltaN']
    M_k = ephemeris['M_0'] + n * sv_position['t_k']
    E_k = M_k
    err = pd.Series(data=[1]*len(sv_position.index))
    i = 0
    while err.abs().min() > 1e-8 and i < 10:
        new_vals = M_k + ephemeris['e']*np.sin(E_k)
        err = new_vals - E_k
        E_k = new_vals
        i += 1
        
    sinE_k = np.sin(E_k)
    cosE_k = np.cos(E_k)
    delT_r = F * ephemeris['e'].pow(ephemeris['sqrtA']) * sinE_k
    delT_oc = transmit_time - ephemeris['t_oc']
    sv_position['delT_sv'] = ephemeris['SVclockBias'] + ephemeris['SVclockDrift'] * delT_oc + ephemeris['SVclockDriftRate'] * delT_oc.pow(2)

    v_k = np.arctan2(np.sqrt(1-ephemeris['e'].pow(2))*sinE_k,(cosE_k - ephemeris['e']))

    Phi_k = v_k + ephemeris['omega']

    sin2Phi_k = np.sin(2*Phi_k)
    cos2Phi_k = np.cos(2*Phi_k)

    du_k = ephemeris['C_us']*sin2Phi_k + ephemeris['C_uc']*cos2Phi_k
    dr_k = ephemeris['C_rs']*sin2Phi_k + ephemeris['C_rc']*cos2Phi_k
    di_k = ephemeris['C_is']*sin2Phi_k + ephemeris['C_ic']*cos2Phi_k

    u_k = Phi_k + du_k

    r_k = A*(1 - ephemeris['e']*np.cos(E_k)) + dr_k

    i_k = ephemeris['i_0'] + di_k + ephemeris['IDOT']*sv_position['t_k']

    x_k_prime = r_k*np.cos(u_k)
    y_k_prime = r_k*np.sin(u_k)

    Omega_k = ephemeris['Omega_0'] + (ephemeris['OmegaDot'] - OmegaDot_e)*sv_position['t_k'] - OmegaDot_e*ephemeris['t_oe']

    sv_position['x_k'] = x_k_prime*np.cos(Omega_k) - y_k_prime*np.cos(i_k)*np.sin(Omega_k)
    sv_position['y_k'] = x_k_prime*np.sin(Omega_k) + y_k_prime*np.cos(i_k)*np.cos(Omega_k)
    sv_position['z_k'] = y_k_prime*np.sin(i_k)
    
    # # Run the function and check out the results:
    # sv_position = calculate_satellite_position(ephemeris, one_epoch['tTxSeconds'])
    # print(sv_position)
    
    return sv_position

In [7]:
def create_position_dataframe(measurements: pd.DataFrame) -> pd.DataFrame:
    position_dataframe = pd.DataFrame()
    epoches = measurements.groupby('Epoch').groups.keys()
    
    for epoch in epoches:
        ephemeris, one_epoch = retrieve_ephemeris_data(measurements, epoch)
        temp_df = calculate_satellite_position(ephemeris, one_epoch['tTxSeconds'])
        position_dataframe = pd.concat([position_dataframe, temp_df])

    return position_dataframe

In [8]:
def remove_duplicates_from_measurements(measurements:pd.DataFrame) -> pd.DataFrame:
    measurements_no_dups = pd.DataFrame()
    epoches = measurements.groupby('Epoch').groups.keys()
    for epoch in epoches:
        temp_df = measurements.loc[(measurements['Epoch'] == epoch) & (measurements['prSeconds'] < 0.1)].drop_duplicates(subset='SvName')
        measurements_no_dups = pd.concat([measurements_no_dups, temp_df])

    return measurements_no_dups

In [14]:
def create_final_dataframe(measurements: pd.DataFrame) -> pd.DataFrame:
    final_df = pd.DataFrame()
    epoches = measurements.groupby('Epoch').groups.keys()
    
    for epoch in epoches:
        ephemeris, one_epoch = retrieve_ephemeris_data(measurements, epoch)
        satellite_coords = calculate_satellite_position(ephemeris, one_epoch['tTxSeconds'])
        
        one_epoch['x_k'] = satellite_coords['x_k']
        one_epoch['y_k'] = satellite_coords['y_k']
        one_epoch['z_k'] = satellite_coords['z_k']
        
        final_df = pd.concat([final_df, one_epoch]) 

    return final_df

In [20]:
measurements, _ = create_dataframes()
measurements = timestamp_generation(measurements)
measurements = pseudorange_calculation(measurements)

df = create_final_dataframe(measurements)

# df = create_position_dataframe(measurements)
# measurements = remove_duplicates_from_measurements(measurements)

In [21]:
df

,utcTimeMillis,TimeNanos,LeapSecond,TimeUncertaintyNanos,FullBiasNanos,BiasNanos,BiasUncertaintyNanos,DriftNanosPerSecond,DriftUncertaintyNanosPerSecond,HardwareClockDiscontinuityCount,...,tRxGnssNanos,GpsWeekNumber,tRxSeconds,tTxSeconds,prSeconds,PrM,PrSigmaM,x_k,y_k,z_k
SvName,,,,,,,,,,,,,,,,,,,,,
G02,1713027078417,332296412000000,18,0.0,-1396730000005342789,-0.769173,34.25081740715541,-137.1602544876685,18.58010122864439,6,...,1.397062e+18,2309.0,579096.417343,579096.340796,0.076547,2.294816e+07,3.297717,1.457321e+07,-5.239568e+06,2.204018e+07
G03,1713027078417,332296412000000,18,0.0,-1396730000005342789,-0.769173,34.25081740715541,-137.1602544876685,18.58010122864439,6,...,1.397062e+18,2309.0,579096.417343,579096.337091,0.080251,2.405878e+07,3.597509,2.318758e+07,-1.212299e+07,4.011163e+06
G08,1713027078417,332296412000000,18,0.0,-1396730000005342789,-0.769173,34.25081740715541,-137.1602544876685,18.58010122864439,6,...,1.397062e+18,2309.0,579096.417343,579096.346849,0.070494,2.113360e+07,2.098547,2.490674e+07,4.694271e+06,8.528082e+06
G10,1713027078417,332296412000000,18,0.0,-1396730000005342789,-0.769173,34.25081740715541,-137.1602544876685,18.58010122864439,6,...,1.397062e+18,2309.0,579096.417343,579096.341329,0.076014,2.278851e+07,4.796679,-1.329165e+06,1.714809e+07,2.035242e+07
G21,1713027078417,332296412000000,18,0.0,-1396730000005342789,-0.769173,34.25081740715541,-137.1602544876685,18.58010122864439,6,...,1.397062e+18,2309.0,579096.417343,579096.344934,0.072409,2.170776e+07,2.997925,1.569971e+07,1.536361e+06,2.191366e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
G10,1713027097417,332315412000000,18,0.0,-1396730000005345219,-0.016075,19.97109575313516,-117.46949555026885,12.486550386904765,6,...,1.397062e+18,2309.0,579115.417343,579115.341306,0.076037,2.279532e+07,5.096472,-1.370345e+06,1.717331e+07,2.032757e+07
G21,1713027097417,332315412000000,18,0.0,-1396730000005345219,-0.016075,19.97109575313516,-117.46949555026885,12.486550386904765,6,...,1.397062e+18,2309.0,579115.417343,579115.344955,0.072387,2.170120e+07,2.398340,1.570767e+07,1.586363e+06,2.190609e+07
G27,1713027097417,332315412000000,18,0.0,-1396730000005345219,-0.016075,19.97109575313516,-117.46949555026885,12.486550386904765,6,...,1.397062e+18,2309.0,579115.417343,579115.343449,0.073894,2.215274e+07,2.698132,2.314381e+07,1.329480e+07,-2.652469e+06


In [27]:
final = df[['UnixTime','Svid','x_k','y_k','z_k','PrM','Cn0DbHz']]
final

,UnixTime,Svid,x_k,y_k,z_k,PrM,Cn0DbHz
SvName,,,,,,,
G02,2024-04-13 16:51:36.417342720+00:00,02,1.457321e+07,-5.239568e+06,2.204018e+07,2.294816e+07,43.0
G03,2024-04-13 16:51:36.417342720+00:00,03,2.318758e+07,-1.212299e+07,4.011163e+06,2.405878e+07,42.0
G08,2024-04-13 16:51:36.417342720+00:00,08,2.490674e+07,4.694271e+06,8.528082e+06,2.113360e+07,47.8
G10,2024-04-13 16:51:36.417342720+00:00,10,-1.329165e+06,1.714809e+07,2.035242e+07,2.278851e+07,39.0
G21,2024-04-13 16:51:36.417342720+00:00,21,1.569971e+07,1.536361e+06,2.191366e+07,2.170776e+07,44.3
...,...,...,...,...,...,...,...
G10,2024-04-13 16:51:55.417345280+00:00,10,-1.370345e+06,1.717331e+07,2.032757e+07,2.279532e+07,38.3
G21,2024-04-13 16:51:55.417345280+00:00,21,1.570767e+07,1.586363e+06,2.190609e+07,2.170120e+07,46.4
G27,2024-04-13 16:51:55.417345280+00:00,27,2.314381e+07,1.329480e+07,-2.652469e+06,2.215274e+07,45.3


In [30]:
final.to_csv('out.csv', index=False)

In [20]:
t1 = measurements.reset_index(drop=True, inplace=False)
t2 = df.reset_index(inplace=False)
count = 0
for i in range(len(t1)):
    if t1.iloc[i].SvName != t2.iloc[i].sv:
        print(t1.iloc[i].SvName,",",t2.iloc[i].sv)
        count+=1
print(count)

G21 , G10
G27 , G21
G28 , G27
G32 , G28
G10 , G32
G21 , G10
G27 , G21
G28 , G27
G32 , G28
G10 , G32
G21 , G10
G27 , G21
G28 , G27
G32 , G28
G10 , G32
G21 , G10
G27 , G21
G28 , G27
G32 , G28
G10 , G32
G21 , G10
G27 , G21
G28 , G27
G32 , G28
G10 , G32
G21 , G10
G27 , G21
G28 , G27
G32 , G28
G10 , G32
G21 , G10
G27 , G21
G28 , G27
G32 , G28
G10 , G32
35


In [14]:
t1['SvName']

0      G02
1      G03
2      G08
3      G10
4      G21
      ... 
155    G10
156    G21
157    G27
158    G28
159    G32
Name: SvName, Length: 160, dtype: object

In [16]:
t2['sv']

0      G02
1      G03
2      G08
3      G10
4      G21
      ... 
155    G10
156    G21
157    G27
158    G28
159    G32
Name: sv, Length: 160, dtype: object

In [26]:
keyss = measurements.groupby('SvName').groups.keys()
for key in keyss:
    print(len(measurements.groupby('SvName').get_group(key)))

20
40
40
29
20
40
29
40


In [27]:
for key in keyss:
    print(len(df.groupby('sv').get_group(key)))

20
20
20
20
20
20
20
20


In [28]:
df

,t_k,delT_sv,x_k,y_k,z_k
sv,,,,,
G02,3096.340796,-0.000454,1.457321e+07,-5.239568e+06,2.204018e+07
G03,3096.337091,0.000309,2.318758e+07,-1.212299e+07,4.011163e+06
G08,3096.346849,0.000120,2.490674e+07,4.694271e+06,8.528082e+06
G10,3112.341329,-0.000008,-1.329165e+06,1.714809e+07,2.035242e+07
G21,3096.344934,0.000127,1.569971e+07,1.536361e+06,2.191366e+07
...,...,...,...,...,...
G10,3131.341306,-0.000008,-1.370345e+06,1.717331e+07,2.032757e+07
G21,3115.344955,0.000127,1.570767e+07,1.586363e+06,2.190609e+07
G27,3115.343449,-0.000018,2.314381e+07,1.329480e+07,-2.652469e+06


In [41]:
temp = measurements.loc[(measurements['Epoch'] == 0) & (measurements['prSeconds'] < 0.1)]# .drop_duplicates(subset='SvName')
for key in temp.groupby('SvName').groups.keys():
    print(len(temp.groupby('SvName').get_group(key)))

1
2
2
1
1
2
1
2


In [52]:
def remove_duplicates_from_measurements(measurements:pd.DataFrame) -> pd.DataFrame:
    measurements_no_dups = pd.DataFrame()
    epoches = measurements.groupby('Epoch').groups.keys()
    for epoch in epoches:
        temp_df = measurements.loc[(measurements['Epoch'] == epoch) & (measurements['prSeconds'] < 0.1)].drop_duplicates(subset='SvName')
        measurements_no_dups = pd.concat([measurements_no_dups, temp_df])

In [53]:
measurements_no_dups

,utcTimeMillis,TimeNanos,LeapSecond,TimeUncertaintyNanos,FullBiasNanos,BiasNanos,BiasUncertaintyNanos,DriftNanosPerSecond,DriftUncertaintyNanosPerSecond,HardwareClockDiscontinuityCount,...,GpsTimeNanos,UnixTime,Epoch,tRxGnssNanos,GpsWeekNumber,tRxSeconds,tTxSeconds,prSeconds,PrM,PrSigmaM
0,1713027078417,332296412000000,18,0.0,-1396730000005342789,-0.769173,34.25081740715541,-137.1602544876685,18.58010122864439,6,...,1.397062e+18,2024-04-13 16:51:36.417342720+00:00,0,1.397062e+18,2309.0,579096.417343,579096.340796,0.076547,2.294816e+07,3.297717
1,1713027078417,332296412000000,18,0.0,-1396730000005342789,-0.769173,34.25081740715541,-137.1602544876685,18.58010122864439,6,...,1.397062e+18,2024-04-13 16:51:36.417342720+00:00,0,1.397062e+18,2309.0,579096.417343,579096.337091,0.080251,2.405878e+07,3.597509
2,1713027078417,332296412000000,18,0.0,-1396730000005342789,-0.769173,34.25081740715541,-137.1602544876685,18.58010122864439,6,...,1.397062e+18,2024-04-13 16:51:36.417342720+00:00,0,1.397062e+18,2309.0,579096.417343,579096.346849,0.070494,2.113360e+07,2.098547
3,1713027078417,332296412000000,18,0.0,-1396730000005342789,-0.769173,34.25081740715541,-137.1602544876685,18.58010122864439,6,...,1.397062e+18,2024-04-13 16:51:36.417342720+00:00,0,1.397062e+18,2309.0,579096.417343,579096.341329,0.076014,2.278851e+07,4.796679
4,1713027078417,332296412000000,18,0.0,-1396730000005342789,-0.769173,34.25081740715541,-137.1602544876685,18.58010122864439,6,...,1.397062e+18,2024-04-13 16:51:36.417342720+00:00,0,1.397062e+18,2309.0,579096.417343,579096.344934,0.072409,2.170776e+07,2.997925
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1013,1713027097417,332315412000000,18,0.0,-1396730000005345219,-0.016075,19.97109575313516,-117.46949555026885,12.486550386904765,6,...,1.397062e+18,2024-04-13 16:51:55.417345280+00:00,19,1.397062e+18,2309.0,579115.417343,579115.341306,0.076037,2.279532e+07,5.096472
1014,1713027097417,332315412000000,18,0.0,-1396730000005345219,-0.016075,19.97109575313516,-117.46949555026885,12.486550386904765,6,...,1.397062e+18,2024-04-13 16:51:55.417345280+00:00,19,1.397062e+18,2309.0,579115.417343,579115.344955,0.072387,2.170120e+07,2.398340
1015,1713027097417,332315412000000,18,0.0,-1396730000005345219,-0.016075,19.97109575313516,-117.46949555026885,12.486550386904765,6,...,1.397062e+18,2024-04-13 16:51:55.417345280+00:00,19,1.397062e+18,2309.0,579115.417343,579115.343449,0.073894,2.215274e+07,2.698132
1016,1713027097417,332315412000000,18,0.0,-1396730000005345219,-0.016075,19.97109575313516,-117.46949555026885,12.486550386904765,6,...,1.397062e+18,2024-04-13 16:51:55.417345280+00:00,19,1.397062e+18,2309.0,579115.417343,579115.337717,0.079626,2.387129e+07,4.796679
